# Cleaning applicants' degree names

In [9]:
import numpy as np
import pandas as pd

from datetime import datetime as dt
import random
import re

In [12]:
data = pd.read_pickle("../../Data/app_opp.pickle")
data.sample(4)

data.degree.unique()

array(["Bachelor's Degree Anticipated 2020",
       "Associate's Degree of Applied Science", 'M.C.A', 'B.C.A', 'HSC',
       'SSLC', 'Associates',
       'Masters of Science in Management Information Systems',
       'Bachelor of Business Administration', 'Oberschule', '1 year ',
       'Certificate',
       "Master's Degree in Industrial Engineering and Economics/Finance",
       'MSN', 'BSN', 'Diploma', 'freshman', 'B.Tech', 'BS',
       'Certificate pending', 'Certificate of Completion',
       'Bachelor of Science in Microbiology', 'Masters of Education',
       'Doctorate in Teaching', 'Masters',
       'Teacher Certification Program', 'Bachelor of Arts in English',
       'Master of Science in Nursing', 'Bachelor of Science in Nursing',
       'Bachelors', 'Bachelor of Science', 'Associate of Science',
       'B.S. in Healthcare Management', "Master's in Special Education",
       "Master's in Political Science", 'Professional',
       "Bachelor's in English", '11', 'High School'

In [13]:
degrees = data[['degree']]

import re

def preprocess_degree(degree):
    # Convert to lowercase
    degree = degree.lower()
    
    # Handle known patterns
    degree = re.sub(r"\bmba\b", 'master of business administration', degree)  # Handle MBA
    degree = re.sub(r"\bbsn\b", 'bachelor of science in nursing', degree)     # Handle B.S.N.
    degree = re.sub(r"bachelor's|bachelor|b\.a\.|b\.s\.|b\.tech", 'bachelor', degree)
    degree = re.sub(r"master's|master|m\.a\.|m\.s\.|m\.tech", 'master', degree)
    degree = re.sub(r"associate's|associate", 'associate', degree)
    degree = re.sub(r"doctorate|ph\.d\.|doctor of", 'doctorate', degree)      # Handle Doctor of
    degree = re.sub(r"certificate|cert\.", 'certificate', degree)
    degree = re.sub(r"diploma|dip\.", 'diploma', degree)
    
    # Remove punctuations
    degree = re.sub(r'[^\w\s]', '', degree)
    
    return degree


def extract_degree_info(degree):
    # Define common degree types
    degree_types = ["bachelor", "master", "associate", "doctorate", "certificate", "diploma", "high school"]
    
    # Extract degree type
    degree_type = "Unknown"
    for d_type in degree_types:
        if d_type in degree:
            degree_type = d_type
            break
            
    # Remove degree type and unnecessary words from the string
    concentration = re.sub(degree_type, '', degree)
    concentration = re.sub(r'\bin\b|\bof\b|\band\b|\bthe\b|\bs\b', '', concentration)
    concentration = concentration.replace(' s ', ' ')  # Handle extra 's'
    concentration = concentration.strip()

    return degree_type, concentration

# Apply preprocessing
degrees['preprocessed_degrees'] = degrees['degree'].apply(preprocess_degree)

# Extract degree info
degrees['degree_type'], degrees['concentration'] = zip(*degrees['preprocessed_degrees'].apply(extract_degree_info))


/var/folders/1w/3dbsn3055ss61n_2tk2w2crm0000gn/T/ipykernel_4687/2743880422.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  degrees['preprocessed_degrees'] = degrees['degree'].apply(preprocess_degree)
/var/folders/1w/3dbsn3055ss61n_2tk2w2crm0000gn/T/ipykernel_4687/2743880422.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  degrees['degree_type'], degrees['concentration'] = zip(*degrees['preprocessed_degrees'].apply(extract_degree_info))
/var/folders/1w/3dbsn3055ss61n_2tk2w2crm0000gn/T/ipykernel_4687

In [14]:
degrees.sample(10)

,degree,preprocessed_degrees,degree_type,concentration
44938,High School,high school,high school,
10163,Bachelor of Business Administration in Accounting,bachelor of business administration in accounting,bachelor,business administration accounting
58444,Diploma/RN,diplomarn,diploma,rn
55636,High School Diploma,high school diploma,diploma,high school
2508,Certificate,certificate,certificate,
19239,High School,high school,high school,
23348,Associate of Science,associate of science,associate,science
33248,Associate of Science,associate of science,associate,science
40952,Master of Education,master of education,master,education
14891,On job Training,on job training,Unknown,on job training


### We can come back to this in order to utilize cleaned data for degrees.